In [ ]:
### Info apie nuotraukas: user_info['edge_owner_to_timeline_media']['edges']

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
import time
import math
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import json
import yaml
import pandas as pd

In [ ]:
from sqlalchemy import create_engine

# Database credentials

CONFIG_PATH = '/home/domantas/config.yml'
conf = yaml.load(open(CONFIG_PATH))

host = conf['mysql']['host']
user = conf['mysql']['username']
passwd = conf['mysql']['password']
db = conf['mysql']['database']

engine = create_engine(f"mysql://{user}:{passwd}@{host}/{db}?charset=utf8mb4")
con = engine.connect()

ig_username = conf['instagram']['username']
ig_password = conf['instagram']['password']

In [ ]:
def get_follow_info(obj):
    fl_id = obj['id']
    fl_username = obj['username']
    fl_full_name = obj['full_name']
    fl_profile_pic_url = obj['profile_pic_url']
    fl_is_private = obj['is_private']
    return {
            'user_id': fl_id, 'username': fl_username, 'full_name': fl_full_name, 'profile_pic': fl_profile_pic_url,
            'is_private': fl_is_private           
            }

# Login Operation

In [ ]:
driver_path = "/home/domantas/webdrivers/geckodriver"
user_agent = '--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1'

profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", user_agent)
driver = webdriver.Firefox(executable_path=driver_path, firefox_profile=profile)
driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')

timeout = 5
# Login
WebDriverWait(driver, 10).until(lambda d: d.find_element_by_xpath("//input[@name='username']"))
username_elem = driver.find_element_by_xpath("//input[@name='username']")
username_elem.send_keys(ig_username)
password_elem = driver.find_element_by_xpath("//input[@name='password']")
password_elem.send_keys(ig_password)
try:
    cookies = driver.find_element_by_xpath("//button[@class='aOOlW  bIiDR  ']").click()
except:
    pass
driver.find_element_by_xpath("//button[@type='submit']").click()
time.sleep(timeout)

In [ ]:
# Get info
user = ''
driver.get(f'https://www.instagram.com/{user}/')

# Meta Info
shared_data = driver.execute_script("return window._sharedData")
user_info = shared_data['entry_data']['ProfilePage'][0]['graphql']['user']

biography = user_info['biography']
public_profile = int(user_info['blocked_by_viewer'])
fb_page = user_info['connected_fb_page'] if user_info['connected_fb_page'] is not None else ''
follows_count = user_info['edge_follow']['count']
followers_count = user_info['edge_followed_by']['count']
posts = user_info['edge_owner_to_timeline_media']['count']
full_name = user_info['full_name']
is_business_account = int(user_info['is_business_account'])
is_private = int(user_info['is_private'])
user_profile_id = user_info['id']
username = user_info['username']

# user_data = {'biography': biography, 'public_profile': public_profile, 'fb_page': fb_page, 'full_name': full_name,
#             'is_business_account': is_business_account, 'is_private': is_private, 'user_id': user_id, 
#              'username': username
#             }
user_data = [user_profile_id, biography, public_profile, fb_page, full_name, is_business_account, is_private, username]

meta_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

end_cursor = shared_data['entry_data']['ProfilePage'][-1]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']


variables = {"id":user_profile_id,"include_reel":True,"fetch_mutual":False,"first":12,"after":end_cursor}
query_hash = "c76146de99bb02f6415203be841dd25a"
url = '''https://www.instagram.com/graphql/query/?query_hash={0}&variables={1}'''.format(query_hash, json.dumps(variables))

r = driver.execute_script('''
    var xmlHttp = new XMLHttpRequest();
    var url = {};
    xmlHttp.open( "GET", url, false);
    xmlHttp.send(null);
    return [xmlHttp.responseText, xmlHttp.status];
'''.format(json.dumps(url))
)
response = r[0]
status_code = r[1]
d = json.loads(response)

users_id = set()
fl_data = []
last_value = len(users_id)
while len(users_id) < followers_count:
    variables = {"id":user_profile_id,"include_reel":True,"fetch_mutual":False,"first":12,"after":end_cursor}
    query_hash = "c76146de99bb02f6415203be841dd25a"
    url = '''https://www.instagram.com/graphql/query/?query_hash={0}&variables={1}'''.format(query_hash, json.dumps(variables))

    r = driver.execute_script('''
        var xmlHttp = new XMLHttpRequest();
        var url = {};
        xmlHttp.open( "GET", url, false);
        xmlHttp.send(null);
        return [xmlHttp.responseText, xmlHttp.status];
    '''.format(json.dumps(url))
    )
    response = r[0]
    status_code = r[1]
    d = json.loads(response)
    for follower in d['data']['user']['edge_followed_by']['edges']:
        if len(users_id) < followers_count:
            followers_data = get_follow_info(follower['node'])
            users_id.add(follower['node']['id'])
            fl_data.append(followers_data)
    
    end_cursor = d['data']['user']['edge_followed_by']['page_info']['end_cursor']
    if last_value != len(users_id):
        last_value = len(users_id)
    else:
        break
    print(status_code, len(users_id), followers_count)
    
# INSERT USER DATA
try:
    sql = '''INSERT INTO user_profile (user_id, biography, public_profile, fb_page, full_name, is_business_account, is_private, username)
        VALUES {};
    '''.format(tuple(user_data))
    con.execute(sql)
    print('User is succesfully added')
except:
    print('User_id already exists in db')

# Inserting data to db

### Update followers info
<br> Tasks to do:
<li> If is_private value set to 0, then make sure that it would exclude it from unfollowed list if that person is not following.
<li> If person which unfollowed you follows again - make sure that status is updated

In [ ]:
df_curr = pd.DataFrame(fl_data)
df_sql = pd.read_sql_query(f'SELECT * from followers where user_profile_id = {user_profile_id};', con=con)  
df_curr.is_private = df_curr.is_private.astype(int)
df_curr['is_following'] = 1
df_curr['user_profile_id'] = user_profile_id
# df_curr.full_name = df_curr.full_name.str.encode('utf-8')
columns = list(set(df_sql.columns) & set(df_curr.columns))

df_mrg = df_sql[columns].merge(df_curr[columns], how='outer', on='user_id', indicator=True)
unfollowed = df_mrg[(df_mrg._merge == 'left_only') & (df_mrg.is_following_x != 0)]['user_id'].to_list()
new_followers = df_mrg[df_mrg._merge == 'right_only']['user_id'].to_list()

#Adding new followers to db
if len(new_followers):
    print('Preparing to add new followers list')
    df_nflwr = df_curr[df_curr.user_id.isin(new_followers)].copy()
    df_nflwr['added_date'] = datetime.now().strftime('%Y-%m-%d %H:%m:%S')
    df_nflwr[df_nflwr.user_id.isin(new_followers)].to_sql('followers', con, if_exists='append', index=False)
    print('Followers list successfully added')
# Change state of no longer followers list
if len(unfollowed):
    print('Preparing to change unfollowed user status')
    sql = '''
        UPDATE followers SET is_following = 0 where user_id IN {};
    '''.format(str(tuple(unfollowed)).replace(',)', ')'))
    con.execute(sql)
    print('Unfollower status list successfully updated')

### New followers

In [ ]:
df_curr[df_curr.user_id.isin(new_followers)]

### Unfollowed

In [ ]:
df_sql[df_sql.user_id.isin(unfollowed)]

### All followers list

In [ ]:
df_curr